<a href="https://colab.research.google.com/github/Akshith213/Technocolabs/blob/main/Toxic_Text_Classification_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pytorch-transformers

In [4]:
from pytorch_transformers import RobertaModel, RobertaTokenizer
from pytorch_transformers import RobertaForSequenceClassification, RobertaConfig

In [8]:
config = RobertaConfig.from_pretrained('roberta-base')

In [9]:
config.num_labels=6

In [10]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)

In [12]:
def prepare_features(seq_1, max_seq_length = 300, 
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [13]:
df=pd.read_csv('/content/drive/MyDrive/Technocolabs/Mini project/train.csv')
test=pd.read_csv('/content/drive/MyDrive/Technocolabs/Mini project/test (1).csv')

In [ ]:
prepare_features(train['comment_text'][0])

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))


In [16]:
df['comment_text']

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [17]:
import nltk
#import speech_recognition as spr
import numpy as np
#import tensorflow
#import tflearn
import random
import json
import pickle
#import pyttsx3
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
import numpy as np
from numpy import dot
from numpy.linalg import norm
import string
import warnings

In [18]:
import nltk
nltk.download('stopwords')
warnings.filterwarnings("ignore")
stop_words=stopwords.words('english')
nltk.download('wordnet')
!pip install contractions
import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
def text_preprocessing(df):
  #df['patterns']=df['patterns'].apply(', '.join)
  df['comment_text'] = df['comment_text'].apply(lambda i:' '.join(i.lower() for i in i.split()))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(i for i in i.split() if not i in stop_words))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(i for i in i.split() if not i in string.punctuation))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(i for i in i.split() if not i.isdigit()))
  df['comment_text']= df['comment_text'].str.replace('[^\w\s]','')
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(WordNetLemmatizer().lemmatize(w) for w in i.split()))
  df['comment_text']=df['comment_text'].apply(lambda i:' '.join(contractions.fix(w) for w in i.split()))
  #return df['comment_text']



In [20]:
text_preprocessing(df)

In [21]:
df['text']=df['comment_text']

In [22]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",explanation edits made username hardcore metal...
1,000103f0d9cfb60f,daww match background colour I am seemingly st...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",daww match background colour I am seemingly st...
2,000113f07ec002fd,hey man I am really trying edit war guy consta...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",hey man I am really trying edit war guy consta...
3,0001b41b1c6bb37e,can not make real suggestion improvement wonde...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",can not make real suggestion improvement wonde...
4,0001d958c54c6e35,you sir hero chance remember page that is on,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",you sir hero chance remember page that is on


In [23]:
train_df,eval_df=train_test_split(df,test_size=0.2)

In [24]:
!pip install simpletransformers

In [32]:
from simpletransformers.classification import MultiLabelClassificationModel

In [35]:
model=MultiLabelClassificationModel('roberta','roberta-base',num_labels=6,args={'train_batch_size':256,'gradient_accumulation_steps':16,'learning_rate':3e-5,'num_train_epochs':3,'max_seq_length':64})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [34]:
train_df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
115637,6a73b419aa8ce4d8,continuity south park section prase jesus stat...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",continuity south park section prase jesus stat...
46049,7b09bce143ee0e1a,lancashire j3mrs deleted every thing I have said,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",lancashire j3mrs deleted every thing I have said
96486,04108acb3df112e9,edit request october request editing hot call ...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",edit request october request editing hot call ...
73535,c4ca9e92332c9d23,put article title,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",put article title
90731,f2c05dc4d614fd02,obviously totally unscientific approach distor...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",obviously totally unscientific approach distor...


In [36]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",explanation edits made username hardcore metal...
1,000103f0d9cfb60f,daww match background colour I am seemingly st...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",daww match background colour I am seemingly st...
2,000113f07ec002fd,hey man I am really trying edit war guy consta...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",hey man I am really trying edit war guy consta...
3,0001b41b1c6bb37e,can not make real suggestion improvement wonde...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",can not make real suggestion improvement wonde...
4,0001d958c54c6e35,you sir hero chance remember page that is on,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",you sir hero chance remember page that is on


In [37]:
model.train_model(train_df)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0ac1873a90>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0b30ec6c50>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    sel

(93, 0.17486122239290947)

In [38]:
eval_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
120938,870dfd28d6fcf958,know really random really hot,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",know really random really hot
86874,e85b5bffeed3dbd6,ip identification please note ip open access w...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",ip identification please note ip open access w...
62084,a61aeaf7a1618afa,corsis apology remark fr article state jerome ...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",corsis apology remark fr article state jerome ...
34890,5d3448d82ed12c3f,dear trulyblue guess voluntary association ask...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",dear trulyblue guess voluntary association ask...
72246,c1699a442bcd3658,I have already accepted two version hob even m...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",I have already accepted two version hob even m...


In [41]:
preds,outputs=model.predict(eval_df['text'].tolist())

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0ac3fdeac8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


In [44]:
len(preds)

31915

In [45]:
eval_df.shape

(31915, 10)

In [71]:
y_test=np.asarray(eval_df['labels'])

In [72]:
y_test

array([(0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0), ...,
       (0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0)],
      dtype=object)

In [55]:
y_pred=[]
for i in range(eval_df.shape[0]):
  y_pred.append(list(preds[i]))

In [73]:
y_test_1=[]
for i in range(eval_df.shape[0]):
  y_test_1.append(list(y_test[i]))

In [74]:
y_test

array([(0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0), ...,
       (0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0)],
      dtype=object)

In [58]:
y_pred[6]

[1, 0, 1, 0, 1, 0]

In [59]:
y_test[6]

(1, 1, 1, 1, 1, 0)

In [64]:
outputs[6]

array([0.92285156, 0.28808594, 0.84667969, 0.14123535, 0.79003906,
       0.21679688])

In [65]:
from sklearn.metrics import hamming_loss

In [77]:
hamming_loss(np.asarray(y_test),np.asarray(y_pred))

ValueError: ignored

In [78]:
import pickle

In [79]:
filename = '/content/drive/MyDrive/Technocolabs/Mini project/model_roberta.sav'

In [80]:
pickle.dump(model, open(filename, 'wb'))

In [81]:
loaded_model = pickle.load(open(filename, 'rb'))

In [83]:
loaded_model

In [84]:
model